In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1298975,2021-05-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1298976,2021-05-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1298977,2021-05-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1298978,2021-05-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
34539,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34541,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34543,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34545,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34547,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
797847,2021-05-22,Arkansas,Arkansas,5001,2119,37.00,5000,Arkansas,AR,Arkansas,State,3017804
797849,2021-05-23,Arkansas,Arkansas,5001,2118,37.00,5000,Arkansas,AR,Arkansas,State,3017804
797851,2021-05-24,Arkansas,Arkansas,5001,2118,37.00,5000,Arkansas,AR,Arkansas,State,3017804
797853,2021-05-25,Arkansas,Arkansas,5001,2118,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1298975,2021-05-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1298976,2021-05-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1298977,2021-05-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1298978,2021-05-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-26') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
491,2021-05-26,Snohomish,Washington,38639,582.00,53061,WA,County,822083,4700.13,70.80
980,2021-05-26,Cook,Illinois,552055,10800.00,17031,IL,County,5150233,10719.03,209.70
1468,2021-05-26,Orange,California,271766,5057.00,6059,CA,County,3175692,8557.69,159.24
1955,2021-05-26,Maricopa,Arizona,548519,10031.00,4013,AZ,County,4485414,12228.95,223.64
2442,2021-05-26,Los Angeles,California,1239945,24212.00,6037,CA,County,10039107,12351.15,241.18
...,...,...,...,...,...,...,...,...,...,...,...
1298199,2021-05-26,Wheeler,Oregon,34,1.00,41069,OR,County,1332,2552.55,75.08
1298425,2021-05-26,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1298620,2021-05-26,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1298811,2021-05-26,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
491,2021-05-26,Snohomish,Washington,38639,582.00,53061,WA,County,822083,4700.13,70.80,70.80,4700.13
980,2021-05-26,Cook,Illinois,552055,10800.00,17031,IL,County,5150233,10719.03,209.70,209.70,10719.03
1468,2021-05-26,Orange,California,271766,5057.00,6059,CA,County,3175692,8557.69,159.24,159.24,8557.69
1955,2021-05-26,Maricopa,Arizona,548519,10031.00,4013,AZ,County,4485414,12228.95,223.64,223.64,12228.95
2442,2021-05-26,Los Angeles,California,1239945,24212.00,6037,CA,County,10039107,12351.15,241.18,241.18,12351.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298199,2021-05-26,Wheeler,Oregon,34,1.00,41069,OR,County,1332,2552.55,75.08,75.08,2552.55
1298425,2021-05-26,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1298620,2021-05-26,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1298811,2021-05-26,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
834316,2021-05-26,Hale,Alabama,2247,78.00,1065,AL,County,14651,15336.84,532.39,532.39,15336.84,1
908261,2021-05-26,Clarke,Alabama,3518,61.00,1025,AL,County,23622,14892.90,258.23,258.23,14892.90,2
652581,2021-05-26,Lowndes,Alabama,1412,54.00,1085,AL,County,9726,14517.79,555.21,555.21,14517.79,3
578543,2021-05-26,Etowah,Alabama,13972,358.00,1055,AL,County,102268,13662.14,350.06,350.06,13662.14,4
485376,2021-05-26,Franklin,Alabama,4271,82.00,1059,AL,County,31362,13618.39,261.46,261.46,13618.39,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230198,2021-05-26,Platte,Wyoming,644,11.00,56031,WY,County,8393,7673.06,131.06,131.06,7673.06,19
796158,2021-05-26,Converse,Wyoming,1013,17.00,56009,WY,County,13822,7328.90,122.99,122.99,7328.90,20
990163,2021-05-26,Lincoln,Wyoming,1412,12.00,56023,WY,County,19830,7120.52,60.51,60.51,7120.52,21
1102299,2021-05-26,Niobrara,Wyoming,162,2.00,56027,WY,County,2356,6876.06,84.89,84.89,6876.06,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
652581,2021-05-26,Lowndes,Alabama,1412,54.00,1085,AL,County,9726,14517.79,555.21,555.21,14517.79,3,1
834316,2021-05-26,Hale,Alabama,2247,78.00,1065,AL,County,14651,15336.84,532.39,532.39,15336.84,1,2
277304,2021-05-26,Walker,Alabama,7282,279.00,1127,AL,County,63521,11463.93,439.22,439.22,11463.93,28,3
576831,2021-05-26,Clay,Alabama,1578,57.00,1027,AL,County,13235,11922.93,430.68,430.68,11922.93,20,4
703818,2021-05-26,Greene,Alabama,928,34.00,1063,AL,County,8111,11441.25,419.18,419.18,11441.25,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702113,2021-05-26,Hot Springs,Wyoming,370,3.00,56017,WY,County,4413,8384.32,67.98,67.98,8384.32,17,19
965442,2021-05-26,Uinta,Wyoming,2263,13.00,56041,WY,County,20226,11188.57,64.27,64.27,11188.57,5,20
990163,2021-05-26,Lincoln,Wyoming,1412,12.00,56023,WY,County,19830,7120.52,60.51,60.51,7120.52,21,21
650019,2021-05-26,Albany,Wyoming,4500,15.00,56001,WY,County,38880,11574.07,38.58,38.58,11574.07,4,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4700.13,21,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4700.13,21,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4700.13,21,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4700.13,21,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4700.13,21,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295937,2021-05-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1295938,2021-05-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1295939,2021-05-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1295940,2021-05-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
830856,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15336.84,2,1,1.00
830857,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15336.84,2,1,0.00
830858,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15336.84,2,1,0.00
830859,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15336.84,2,1,0.00
830860,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15336.84,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006767,2021-05-22,Crook,Wyoming,438,12.00,56011,WY,County,7584,5775.32,158.23,158.23,5814.87,6,23,0.00
1006768,2021-05-23,Crook,Wyoming,438,12.00,56011,WY,County,7584,5775.32,158.23,158.23,5814.87,6,23,0.00
1006769,2021-05-24,Crook,Wyoming,438,12.00,56011,WY,County,7584,5775.32,158.23,158.23,5814.87,6,23,0.00
1006770,2021-05-25,Crook,Wyoming,440,12.00,56011,WY,County,7584,5801.69,158.23,158.23,5814.87,6,23,2.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
649967,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,1.00,0.00
649968,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
649969,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
649970,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
649971,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274238,2021-05-22,Teton,Wyoming,3776,9.00,56039,WY,County,23464,16092.74,38.36,38.36,16114.05,23,1,0.00,0.00
274239,2021-05-23,Teton,Wyoming,3776,9.00,56039,WY,County,23464,16092.74,38.36,38.36,16114.05,23,1,0.00,0.00
274240,2021-05-24,Teton,Wyoming,3780,9.00,56039,WY,County,23464,16109.79,38.36,38.36,16114.05,23,1,4.00,0.00
274241,2021-05-25,Teton,Wyoming,3780,9.00,56039,WY,County,23464,16109.79,38.36,38.36,16114.05,23,1,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-26') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
331088,2021-05-26,Imperial,California,28586,735.00,6025,CA,County,181215,15774.63,405.60,405.60,15774.63,1,2,11.00,0.00
2442,2021-05-26,Los Angeles,California,1239945,24212.00,6037,CA,County,10039107,12351.15,241.18,241.18,12351.15,2,5,187.00,11.00
164758,2021-05-26,San Bernardino,California,297848,4755.00,6071,CA,County,2180085,13662.22,218.11,218.11,13662.22,3,4,160.00,8.00
580207,2021-05-26,Inyo,California,1429,38.00,6027,CA,County,18039,7921.73,210.65,210.65,7921.73,4,25,0.00,0.00
73625,2021-05-26,Stanislaus,California,62680,1066.00,6099,CA,County,550660,11382.70,193.59,193.59,11382.70,5,9,33.00,0.00
34092,2021-05-26,Riverside,California,300550,4608.00,6065,CA,County,2470546,12165.33,186.52,186.52,12165.33,6,7,74.00,0.00
58565,2021-05-26,San Joaquin,California,73758,1419.00,6077,CA,County,762148,9677.65,186.18,186.18,9677.65,7,17,41.00,0.00
91302,2021-05-26,Tulare,California,50122,843.00,6107,CA,County,466195,10751.30,180.83,180.83,10751.30,8,10,-12.00,1.00
33200,2021-05-26,Fresno,California,102329,1701.00,6019,CA,County,999101,10242.11,170.25,170.25,10242.11,9,14,40.00,3.00
525741,2021-05-26,Merced,California,32110,468.00,6047,CA,County,277680,11563.67,168.54,168.54,11563.67,10,8,17.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1229749,2021-05-26,Lassen,California,5726,24.00,6035,CA,County,30573,18728.94,78.50,78.50,18728.94,35,1,5.00,0.00
331088,2021-05-26,Imperial,California,28586,735.00,6025,CA,County,181215,15774.63,405.60,405.60,15774.63,1,2,11.00,0.00
703760,2021-05-26,Kings,California,23047,247.00,6031,CA,County,152940,15069.31,161.50,161.50,15069.31,11,3,5.00,0.00
164758,2021-05-26,San Bernardino,California,297848,4755.00,6071,CA,County,2180085,13662.22,218.11,218.11,13662.22,3,4,160.00,8.00
2442,2021-05-26,Los Angeles,California,1239945,24212.00,6037,CA,County,10039107,12351.15,241.18,241.18,12351.15,2,5,187.00,11.00
207189,2021-05-26,Kern,California,109830,1385.00,6029,CA,County,900202,12200.59,153.85,153.85,12200.59,14,6,92.00,0.00
34092,2021-05-26,Riverside,California,300550,4608.00,6065,CA,County,2470546,12165.33,186.52,186.52,12165.33,6,7,74.00,0.00
525741,2021-05-26,Merced,California,32110,468.00,6047,CA,County,277680,11563.67,168.54,168.54,11563.67,10,8,17.00,0.00
73625,2021-05-26,Stanislaus,California,62680,1066.00,6099,CA,County,550660,11382.70,193.59,193.59,11382.70,5,9,33.00,0.00
91302,2021-05-26,Tulare,California,50122,843.00,6107,CA,County,466195,10751.30,180.83,180.83,10751.30,8,10,-12.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
331088,2021-05-26,Imperial,California,28586,735.00,6025,CA,County,181215,15774.63,405.60,405.60,15774.63,1,2,11.00,0.00
2442,2021-05-26,Los Angeles,California,1239945,24212.00,6037,CA,County,10039107,12351.15,241.18,241.18,12351.15,2,5,187.00,11.00
164758,2021-05-26,San Bernardino,California,297848,4755.00,6071,CA,County,2180085,13662.22,218.11,218.11,13662.22,3,4,160.00,8.00
580207,2021-05-26,Inyo,California,1429,38.00,6027,CA,County,18039,7921.73,210.65,210.65,7921.73,4,25,0.00,0.00
73625,2021-05-26,Stanislaus,California,62680,1066.00,6099,CA,County,550660,11382.70,193.59,193.59,11382.70,5,9,33.00,0.00
34092,2021-05-26,Riverside,California,300550,4608.00,6065,CA,County,2470546,12165.33,186.52,186.52,12165.33,6,7,74.00,0.00
58565,2021-05-26,San Joaquin,California,73758,1419.00,6077,CA,County,762148,9677.65,186.18,186.18,9677.65,7,17,41.00,0.00
91302,2021-05-26,Tulare,California,50122,843.00,6107,CA,County,466195,10751.30,180.83,180.83,10751.30,8,10,-12.00,1.00
33200,2021-05-26,Fresno,California,102329,1701.00,6019,CA,County,999101,10242.11,170.25,170.25,10242.11,9,14,40.00,3.00
525741,2021-05-26,Merced,California,32110,468.00,6047,CA,County,277680,11563.67,168.54,168.54,11563.67,10,8,17.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,405.60,15774.63,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,405.60,15774.63,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,405.60,15774.63,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,405.60,15774.63,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,405.60,15774.63,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4342,05/22/21,Lassen,5721,24.00,18712.59,78.50,78.50,18728.94,35,1,1.00,0.00
4343,05/23/21,Lassen,5721,24.00,18712.59,78.50,78.50,18728.94,35,1,0.00,0.00
4344,05/24/21,Lassen,5721,24.00,18712.59,78.50,78.50,18728.94,35,1,0.00,0.00
4345,05/25/21,Lassen,5721,24.00,18712.59,78.50,78.50,18728.94,35,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,405.60,15774.63,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,405.60,15774.63,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,405.60,15774.63,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,405.60,15774.63,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,405.60,15774.63,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4428,05/22/21,Merced,32028,468.00,11534.14,168.54,168.54,11563.67,10,8,0.00,0.00
4429,05/23/21,Merced,32028,468.00,11534.14,168.54,168.54,11563.67,10,8,0.00,0.00
4430,05/24/21,Merced,32082,468.00,11553.59,168.54,168.54,11563.67,10,8,54.00,0.00
4431,05/25/21,Merced,32093,468.00,11557.55,168.54,168.54,11563.67,10,8,11.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)